In [ ]:
from aiida import load_profile

load_profile();

In [ ]:
%%html

<style>
  .output_subarea {
    max-width: none !important;
  }
  .header-tr {
    column-gap: 5px;
  }
  .th {
    padding: 2px 5px;
    font-weight: bold;
    color: #fff;
    text-align: center;
    background-color: #000;
  }
  .tr:hover {
    background-color: #f5f5f5;
  }
  hr {
    border-top: 1px solid #d3d3d3;
  }
</style>


In [ ]:
import ipywidgets as ipw


class LoadingWidget(ipw.HBox):
    """Widget for displaying a loading spinner."""

    def __init__(self, message="Loading", **kwargs):
        super().__init__(
            children=[
                ipw.Label(message),
                ipw.HTML(
                    value="<i class='fa fa-spinner fa-spin fa-2x fa-fw'/>",
                    layout=ipw.Layout(margin="12px 0 6px"),
                ),
            ],
            layout=ipw.Layout(
                justify_content="center",
                align_items="center",
                **kwargs.pop("layout", {}),
            ),
            **kwargs,
        )
        self.add_class("loading")

# AiiDAlab code setup

This page provides an interface for creating and managing AiiDA codes on local or remote machines.


<hr>

## Create new code

To create a new AiiDA code for use in a calculation, follow these steps:

#### Quick setup (using [AiiDA resource registry](https://github.com/aiidateam/aiida-resource-registry) recipes)

1. Select the domain of your remote machine
2. Select the computer recipe
3. Select the code recipe
4. Complete the remaining fields (these depend on the combination selected in steps 1-3)
5. Click **Quick setup**

#### Manual setup (create new resources from scratch)

1. Tick the checkbox above the **Quick setup** button
2. Follow along with the provided instructions

Newly created codes will be available in the **Available codes** section, as well as in code selectors provided in various apps.


In [ ]:
new_codes_widget_container = ipw.VBox(
    children=[
        LoadingWidget("Loading resource setup widget"),
    ],
)
display(new_codes_widget_container)

<hr>

## Available codes

Below are the available codes in the AiiDA database. Use the controls to filter the list by label, executable path, and visibility. You can mark individual codes as _hidden_ to exclude them from code selectors used in apps (note that some apps may enforce the inclusion of hidden codes). Click **Unhide all** to include all codes. If codes are deleted from the database, click **Refresh codes** to reflect the change in the list.


In [ ]:
available_codes_widget_container = ipw.VBox(
    children=[
        LoadingWidget("Loading available codes"),
    ],
)
display(available_codes_widget_container)

In [ ]:
import ipywidgets as ipw
from aiidalab_widgets_base.computational_resources import ResourceSetupBaseWidget
from aiidalab_widgets_base.utils import StatusHTML

setup_message = StatusHTML(clear_after=15)

widget = ResourceSetupBaseWidget()
ipw.dlink(
    (widget, "message"),
    (setup_message, "message"),
)

new_codes_widget_container.children = [
    widget,
    setup_message,
]

In [ ]:
from home.code_setup import create_paginated_table, fetch_code_data

output = ipw.Output()


def render():
    data = fetch_code_data()
    paginated_table = create_paginated_table(data)
    output.clear_output()
    with output:
        display(paginated_table)


refresh_button = ipw.Button(
    description="Refresh codes",
    button_style="primary",
    icon="refresh",
    tooltip="Refresh the list of codes",
    layout=ipw.Layout(width="fit-content"),
)
refresh_button.on_click(lambda _: render())

available_codes_widget_container.children = [
    refresh_button,
    output,
]

render()

In [ ]:
def on_code_setup_message_change(change):
    if "created" in change["new"]:
        render()


widget.aiida_code_setup.observe(
    on_code_setup_message_change,
    "message",
)